In [ ]:
#Indexing Part:

In [ ]:
#The following 2 code blocks are using small file set (100mb):

In [ ]:
#Inverted Index using mapreduce but not using cluster:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os, re
from time import *

begin_time = time()    #to get the running time

#spark = SparkSession.builder.master('spark://dpw2frankm:7077').appName('SparkInvIndex').getOrCreate()
#sc = spark.sparkContext
sc = SparkContext.getOrCreate(SparkConf())

inverted_index = sc.wholeTextFiles('hdfs://localhost:9000/user/root/test_small/')\
                    .flatMap(lambda x: [((os.path.basename(x[0]).split(".")[0] ,i) ,1) \
                                        for i in re.split('\\W', x[1])])\
                    .reduceByKey(lambda a, b: a + b)\
                    .map(lambda x: (x[0][1],(x[0][0],x[1])))

#we skip the collect and print step because it's too slow:
output = inverted_index.collect()
for i in range(inverted_index.count()): 
    
    print(output[i])
    #if i>10:   #limit number to print
     #   break

end_time = time()    #to get the running time
run_time = end_time - begin_time    
print('Running time: ', run_time)

In [ ]:
#Inverted Index using cluster and mapreduce:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os, re
from time import *

begin_time = time()    #to get the running time

spark = SparkSession.builder.master('spark://dpw2frankm:7077').appName('SparkInvIndex').getOrCreate()
sc = spark.sparkContext
#sc = SparkContext.getOrCreate(SparkConf())

inverted_index = sc.wholeTextFiles('hdfs://localhost:9000/user/root/test_small/')\
                    .flatMap(lambda x: [((os.path.basename(x[0]).split(".")[0] ,i) ,1) \
                                        for i in re.split('\\W', x[1])])\
                    .reduceByKey(lambda a, b: a + b)\
                    .map(lambda x: (x[0][1],(x[0][0],x[1])))

#output = inverted_index.collect()
#for i in range(inverted_index.count()): 
    
 #   print(output[i])
  #  if i>10:   #limit number to print
   #     break

end_time = time()    #to get the running time
run_time = end_time - begin_time
print('Running time: ', run_time)

In [ ]:
#The following 2 code blocks are using large file set (1gb):

In [ ]:
#Inverted Index using mapreduce but not using cluster:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os, re
from time import *

begin_time = time()    #to get the running time

#spark = SparkSession.builder.master('spark://dpw2frankm:7077').appName('SparkInvIndex').getOrCreate()
#sc = spark.sparkContext
sc = SparkContext.getOrCreate(SparkConf())

inverted_index = sc.wholeTextFiles('hdfs://localhost:9000/user/root/ChildrenBooks/')\
                    .flatMap(lambda x: [((os.path.basename(x[0]).split(".")[0] ,i) ,1) \
                                        for i in re.split('\\W', x[1])])\
                    .reduceByKey(lambda a, b: a + b)\
                    .map(lambda x: (x[0][1],(x[0][0],x[1])))

#we skip the collect and print step because it's too slow:
output = inverted_index.collect()
for i in range(inverted_index.count()): 
    
    print(output[i])
    #if i>100:   #limit number to print
     #   break

end_time = time()    #to get the running time
run_time = end_time - begin_time    
print('Running time: ', run_time)

In [ ]:
print('Running time: ', run_time)

In [ ]:
#Inverted Index using cluster and mapreduce:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os, re
from time import *

begin_time = time()    #to get the running time 

spark = SparkSession.builder.master('spark://dpw2frankm:7077').appName('SparkInvIndex').getOrCreate()
sc = spark.sparkContext
#sc = SparkContext.getOrCreate(SparkConf())

inverted_index = sc.wholeTextFiles('hdfs://localhost:9000/user/root/ChildrenBooks/')\
                    .flatMap(lambda x: [((os.path.basename(x[0]).split(".")[0] ,i) ,1) \
                                        for i in re.split('\\W', x[1])])\
                    .reduceByKey(lambda a, b: a + b)\
                    .map(lambda x: (x[0][1],(x[0][0],x[1])))

#output = inverted_index.collect()
#for i in range(inverted_index.count()): 
    
 #   print(output[i])
  #  if i>100:   #limit number to print
   #     break

end_time = time()    #to get the running time
run_time = end_time - begin_time
print('Running time: ', run_time)

In [ ]:
#Ranking Part:

In [ ]:
#Ranking with TF-IDF:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
import os, re
import math

#First we compute the TF-IDF of all files in ./iitfidf, save as an RDD
#We need SQLContext for processing RDD<->DF
#If you want to run the job on Spark cluster, you have to modify the following line, e.g.:
spark = SparkSession.builder.master('spark://dpw2tcxu:7077').appName('MyWordCount').getOrCreate()
sc = spark.sparkContext
#sc = SparkContext.getOrCreate(SparkConf())
sql = SQLContext(sc)

#If you want to run the job with Hadoop HDFS, you have to modify the following line:
data = sc.wholeTextFiles('hdfs://localhost:9000/user/root/ChildrenBooks/')

numFiles = data.count()  

wordcount = data.flatMap(lambda x: [((os.path.basename(x[0]) ,i) ,1) for i in re.split('\\W', x[1])])\
        .reduceByKey(lambda a, b: a + b)

tf = wordcount.map(lambda x: (x[0][1],(x[0][0],x[1])))

idf = wordcount.map(lambda x: (x[0][1], (x[0][0], x[1], 1)))\
        .map(lambda x: (x[0], x[1][2]))\
        .reduceByKey(lambda x, y: x + y)\
        .map(lambda x: (x[0], math.log10(numFiles / x[1])))

#Slightly modified map output as (doc, (term, tfidf))
tfidf = tf.join(idf)\
            .map(lambda x: (x[1][0][0], (x[0], x[1][0][1] * x[1][1])))\
            .sortByKey()

#Then we convert the TF-IDF to an DF, and save to the disk
lines = tfidf.map(lambda x: (x[0], x[1][0], x[1][1])).toDF()
lines.write.save("tfidf-index")

In [ ]:
#The RDD is mapped as map(lambda x: (x['term'],(x['doc'],x['tfidf']))), for example:
#tfidf_RDD = tfidf.map(lambda x: (x[1][0],(x[0],x[1][1])))

#Now we load the TF-IDF index from the disk, can convert it to an RDD, and map as x['term'],(x['doc'],x['tfidf'])
tfidf_RDD = sql.read.parquet("tfidf-index").rdd.map(lambda x: (x['_2'],(x['_1'],x['_3'])))

def tokenize(s):
  return re.split("\\W+", s.lower())

def search(query, topN):
  tokens = sc.parallelize(tokenize(query)).map(lambda x: (x, 1) ).collectAsMap()
  bcTokens = sc.broadcast(tokens)

  #connect to documents with terms in the Query. to Limit the computation space
  #so that we don't attempt to compute similarity for docs that have no words in common with our query.  
  joined_tfidf = tfidf_RDD.map(lambda x: (x[0], bcTokens.value.get(x[0], '-'), x[1]) ).filter(lambda x: x[1] != '-' )
  
  #compute the score using aggregateByKey
  scount = joined_tfidf.map(lambda a: a[2]).aggregateByKey((0,0),
  (lambda acc, value: (acc[0] + value, acc[1] + 1)),
  (lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])) )
  
  scores = scount.map(lambda x: ( x[1][0]*x[1][1]/len(tokens), x[0]) ).top(topN)
  
  return scores

search("Little Red Riding-Hood", 5)